In [1]:
import sklearn.feature_extraction.text as skf
import sklearn.metrics.pairwise as skd
import sklearn.model_selection as skm
import sklearn.metrics as skms

from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.decomposition import TruncatedSVD
from tqdm.auto import tqdm
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords

import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 1000)

In [2]:
ru_stopwords = stopwords.words("russian")

In [3]:
data = pd.read_csv("./labeled.csv")

In [4]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)\n",1.0


In [5]:
data['toxic'].value_counts()

0.0    9586
1.0    4826
Name: toxic, dtype: int64

In [6]:
data.shape

(14412, 2)

In [7]:
data['comment'] = data['comment'].str.replace("\n"," ")

In [8]:
X, y = data['comment'].tolist(), data['toxic'].values.squeeze()

## Task 2

In [9]:
%%time
vec = skf.TfidfVectorizer(ngram_range=(1, 3), min_df=10)
svd = TruncatedSVD(n_components=100, n_iter=100, random_state=42) 
# без нулей так без нулей
X_vec = vec.fit_transform(X)
X_vec = svd.fit_transform(X_vec)

CPU times: user 45.4 s, sys: 1min 40s, total: 2min 25s
Wall time: 22.6 s


### a - Посчитайте близость между 3 и 12666 текстами в датасете (labeled.csv из семинара) 

In [10]:
skd.cosine_similarity([X_vec[2]], [X_vec[12665]])

array([[0.16230719]])

### б - найдите 3 самых близких текста к тексту номер 43; выведите сами тексты и значения близостей, а не только индексы этих текстов. 

In [11]:
def k_nearest(idx: int, k: int = 3):
    sims = np.array(list(map(
        lambda x: skd.cosine_similarity([x], [X_vec[idx]]), tqdm(X_vec)
    ))).squeeze()
    
    nearest = np.argsort(sims, kind='heapsort')[-k-1:] ## search for k+1 neighbors to skip first as it is our vector
    result = []
    for n_idx in nearest[::-1][1:]: # to skip first result
        result.append({'idx': n_idx, 'similarity': sims[n_idx], 'text': X[n_idx]})
    return result

In [12]:
pd.DataFrame(k_nearest(42, k=3)) # dataframe for visualization proposes

,idx,similarity,text
0,570,0.669467,"он, как любой психически здоровый человек Он и в существование психики не верит. Что ты на это скажешь, девственник?"
1,3719,0.632034,"Можно же совмещать:3Быть молодым, целеустремлённым, ленным, полным надежд глупцом:3"
2,5727,0.624819,"Рак ебучий, как же ты заебал, сука. СДОХНИ БЛЯДЬ И ПЕРЕСТАНЬ ТАСКАТЬ НА ФИГУРЧ ЗАЛЕТУХ!!1"


## Task 3

In [13]:
X_train, X_test, y_train, y_test = skm.train_test_split(np.array(X), y, test_size=0.2, random_state=1993)

#### Первая модель - Naive Bayes + CountVectorizer

In [14]:
nb = Pipeline([
    (
        "vec",
        skf.CountVectorizer(
            lowercase=True,
            max_df=0.9, 
            max_features=10000,
            min_df=2,
            ngram_range=(1,1),
            stop_words=None
        )
    ),
    (
        "clf",
        MultinomialNB(alpha=0.9, fit_prior=False)
    )
])

In [15]:
nb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vec',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.9,
                                 max_features=10000, min_df=2,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=0.9, class_prior=None, fit_prior=False))],
         verbose=False)

In [16]:
print(skms.classification_report(y_test, nb.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.90      0.89      0.89      1926
         1.0       0.78      0.80      0.79       957

    accuracy                           0.86      2883
   macro avg       0.84      0.84      0.84      2883
weighted avg       0.86      0.86      0.86      2883



#### Вторая модель - LogReg + TF-IDF

In [17]:
logreg = Pipeline([
    (
        'vec',
        skf.TfidfVectorizer(
            stop_words = None,
            ngram_range = (1, 1),
            min_df = 1,
            max_df = 1.0,
            max_features = 75000,
            lowercase = True
        )
    ),
    (
        'clf',
        LogisticRegression(
            C = 100.0,
            penalty = 'l2',
            fit_intercept = True
        )
    )
])

In [18]:
logreg.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vec',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=75000,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LogisticRegression(C=100.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scalin

In [19]:
print(skms.classification_report(y_test, logreg.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.87      0.94      0.90      1926
         1.0       0.86      0.71      0.78       957

    accuracy                           0.86      2883
   macro avg       0.86      0.83      0.84      2883
weighted avg       0.86      0.86      0.86      2883



### Выводим топ-10 самых токсичных текстов по версии каждой из моделей

In [20]:
# sort by probability of toxic class
top_nb = np.argsort(nb.predict_proba(X)[:, 1])[-10:]
top_logreg = np.argsort(logreg.predict_proba(X)[:, 1])[-10:]

Из приведенных текстов видно, что модели на основе CountVectorizer + Naive Bayes требуется значительно большее количество токсичных слов для того чтобы присвоить тексту класс токсичности(или высокой вероятности оной). В то же время модель на основе TF-IDF + LogReg хорошо справляетс и с маленькими текстами. Субъективно  - можно заметить что тексты 3 и 4 из модели NB не являются в действительности токсичными, однако содержат очень много слов(навальный, соловьев, дота и т.д), вероятнее всего употребляющихся в токсичных контекстах в корпусе, видимо поэтому они и были промаркированы как токсичные.

In [21]:
X = np.array(X)
for i, (text_nb, text_logreg) in enumerate(zip(X[top_nb], X[top_logreg])):
    print(("="*50) + f"[{i}]" + ("="*50))
    print("--------[NB-scored text]--------")
    print(text_nb)
    print("------[logreg-scored text]------")
    print(text_logreg)

==================================================[0]==================================================
--------[NB-scored text]--------
эта мразота терроризирует треды более двух лет, вниманиеблядствует и потешается над анонами. Пруфы того, что в треды срет один и тот же человек, потешается над анонами и злоупотребляет бездействием модераторов . эти вебмки сделаны им Его тред, куда он срал своими скримерами поискав по метадате, можно найти десятки тредов, куда он срал своими скримерами, по 15-20 штук в треде. Пруфы засирания тредов , к примеру. Поиском можно найти намного больше 2 вебм начало 1:19 0:49 и 3:57 3:16 1:23 Пруфы многократных жалоб от анонов на деятельность этого серуна ПРИМИТЕ МЕРЫ БЛЯДЬ!!!!!! 
------[logreg-scored text]------
Какие же хохлы революционеры, пиздец просто 
==================================================[1]==================================================
--------[NB-scored text]--------
именно так. смотрю на тебя, стаса, вестника дури и считаю ебланами.